# Simulation template 

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963). Here the simulation is run with features generated from fourier components in the first scenario. This scenario is characterized by using only the EVEN wavevector space eigenmodes, computed from ALL real space components.

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 766 ms, sys: 255 ms, total: 1.02 s
Wall time: 586 ms


In [2]:
%%time
### Dataset and simulation parameters
#csv_path = SSH2_PERIODIC_100_6561_CSV 
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
#allowed_windings = [-1,0,1,2]
simulation_dir = SSH1_PERIODIC_1ST_SCENARIO_100_6561_SIMULATION_DIR
#simulation_dir = SSH2_PERIODIC_ENGINEERED_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None
shuffle_features = False
random_state = 763

### Fourier features
fourier_mode = "dft"
fourier_features_to_use = None
fourier_real = "real"
fourier_normalize = True
fourier_fillna = False

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
#mode=mode
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    40,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
]
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_100_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_1st_scenario_100_6561"
random_state = 244854
fourier_features_to_use = None
fourier_mode = None
fourier_real = None
fourier_normalize = None
fourier_fillna = None


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,fourier_mode,fourier_features_to_use,fourier_real,\
                       fourier_normalize, fourier_fillna , random_state)
#simulation = Simulation(**dict_args)


#(self, csv_path, model_name, model_kw, allowed_windings, simulation_dir = None, val_split = 0, 
# features_to_use = None, shuffle_features = False, fourier_mode=None, fourier_features_to_use=None,
# fourier_real = False, fourier_normalize = False, Fourier_fillna=False, random_state = None):



print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Columns: 108 entries, id to feat99
dtypes: float64(103), int32(3), object(2)
memory usage: 538.1+ MB
CPU times: user 5.03 s, sys: 324 ms, total: 5.35 s
Wall time: 5.35 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130965,0.301130,...,-0.080279,-0.115687,-0.174988,0.245064,0.319763,-0.059082,-0.000556,-0.245133,-0.301130,0.130965
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130965,-0.301130,...,-0.080279,0.115687,-0.174988,-0.245064,0.319763,0.059082,-0.000556,0.245133,-0.301130,-0.130965
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.148414,-0.042467,...,0.185442,-0.080620,-0.066211,-0.121339,-0.151700,0.192513,0.191642,-0.050956,-0.042467,-0.148414
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.069630,-0.026147,...,0.102009,-0.100671,-0.081002,-0.020715,-0.058717,0.115834,0.118274,-0.054666,-0.026147,-0.069630
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.010206,-0.014625,...,0.047555,-0.041368,-0.018852,-0.017673,-0.023377,0.031447,0.010280,0.000925,0.014625,-0.010206


####  Fourier features

In [5]:
%%time
#simulation.generate_fourier_df()
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.info() 
    simulation.fourier_dataframe.sample(15)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [6]:
print("features_to_use: ")
print("\n")
print(simulation.features_to_use)
if simulation.features_to_use is not None:
    print("Number of real space features:")
    print("\n")
    print(len(simulation.features_to_use))
print("\n")
print("fourier_features: ")
print("\n")
print(simulation.fourier_features)
print("\n")
if simulation.fourier_features is not None:
    print("Number of Fourier features: ")
    print("\n")
    print(len(simulation.fourier_features))

features_to_use: 


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Number of real space features:


25


fourier_features: 


None




In [7]:
#simulation.dataframe.iloc[331456,:]

In [8]:
#for ix, obj in enumerate(simulation.dataframe.iloc[331456,:].values):
#    print(ix,obj)

In [9]:
#simulation.fourier_engineered_dataframe.dct_feat0

In [10]:
#simulation.features

In [11]:
#simulation.fourier_matrix

In [12]:
#1/simulation.n_features*simulation.dataframe.loc[:,simulation.features].sum(axis=1)

In [13]:
#df_2 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==2]
#df_2.describe()

In [14]:
#df_1 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==1]
#df_1.describe()

In [15]:
#df_0 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==0]
#df_0.describe()

In [16]:
#plt.figure(figsize=(10,10))
#plt.scatter(df_1.fourier_feat0,df_1.fourier_feat2)
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [17]:
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [18]:
#plt.scatter(df_2.fourier_feat0,df_2.fourier_feat2)

In [19]:
#df_1.hist(figsize=(15,15))

In [20]:
#df_0.hist(figsize=(15,15))

#### Checking initialization

In [21]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  100
n_hamiltonians:  6561
n_ts:  2
CPU times: user 1.27 ms, sys: 134 µs, total: 1.4 ms
Wall time: 736 µs


In [22]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [23]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params,shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

running experiments:   1%|          | 1/100 [00:22<37:08, 22.51s/it]

running experiments:   2%|▏         | 2/100 [00:45<37:02, 22.68s/it]

running experiments:   3%|▎         | 3/100 [01:08<36:39, 22.68s/it]

running experiments:   4%|▍         | 4/100 [01:31<36:23, 22.74s/it]

...

running experiments:  98%|█████████▊| 98/100 [37:46<00:45, 22.88s/it]

running experiments:  99%|█████████▉| 99/100 [38:09<00:22, 22.74s/it]

running experiments: 100%|██████████| 100/100 [38:31<00:00, 22.66s/it]

running experiments: 100%|██████████| 100/100 [38:31<00:00, 23.12s/it]

CPU times: user 4h 9min 13s, sys: 14.8 s, total: 4h 9min 28s
Wall time: 38min 31s


In [24]:
simulation.dataframe.head(10)

,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130965,0.301130,...,-0.080279,-0.115687,-0.174988,0.245064,0.319763,-0.059082,-0.000556,-0.245133,-0.301130,0.130965
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130965,-0.301130,...,-0.080279,0.115687,-0.174988,-0.245064,0.319763,0.059082,-0.000556,0.245133,-0.301130,-0.130965
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.148414,-0.042467,...,0.185442,-0.080620,-0.066211,-0.121339,-0.151700,0.192513,0.191642,-0.050956,-0.042467,-0.148414
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.069630,-0.026147,...,0.102009,-0.100671,-0.081002,-0.020715,-0.058717,0.115834,0.118274,-0.054666,-0.026147,-0.069630
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,0.010206,-0.014625,...,0.047555,-0.041368,-0.018852,-0.017673,-0.023377,0.031447,0.010280,0.000925,0.014625,-0.010206
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.116962,0.021475,...,-0.086901,-0.045676,-0.028034,0.129266,0.130083,-0.098190,-0.104188,-0.007911,0.021475,0.116962
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.037175,-0.095882,...,-0.027490,0.046168,0.063419,-0.041630,-0.085769,-0.010604,-0.009822,0.084024,0.095882,-0.037175
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,0.027709,-0.038180,...,0.123344,-0.114320,-0.057118,-0.040291,-0.061263,0.087153,0.029599,-0.001276,0.038180,-0.027709
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.059795,0.030272,...,-0.082347,0.140996,0.128386,-0.036608,0.029628,-0.124877,-0.140478,0.088573,0.030272,0.059795
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.141605,0.013497,...,-0.035718,-0.110014,-0.082452,0.146084,0.152588,-0.067575,-0.087187,-0.043484,0.013497,0.141605


#### Viewing a random sample

In [25]:
%%time
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.sample(frac=0.1, replace=False)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.05 µs


#### Checking train/val/test splits again

In [26]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [479, 1199, 2058, 6287, 6483, 3435, 5873, 4628, 2333, 3621, 4553, 2119, 1165, 2978, 3888, 4241, 2745, 4847, 3045, 1454, 5122, 4765, 5019, 2481, 1006, 1180, 1613, 345, 2257, 6105, 5241, 1689, 2695, 5274, 2430, 3176, 3828, 641, 4149, 6464, 3527, 5488, 3687, 4173, 2286, 2732, 2903, 2472, 2276, 3052, 3895, 4712, 3521, 6366, 4955, 5112, 3369, 1765, 2825, 2736, 4176, 1005, 2650, 4272, 3166, 4743, 792, 2689, 1621, 6518, 4774, 2234, 5094, 1079, 1420, 4969, 4219, 4967, 5020, 1775, 4784, 3821, 1210, 4418, 6140, 800, 6084, 613, 3459, 4061, 5837, 4190, 1488, 3214, 1525, 5473, 684, 4853, 833, 6382, 5892, 3960, 5693, 5265, 5129, 6265, 389, 5691, 4017, 3489, 4242, 5998, 3662, 2731, 5430, 6440, 5263, 2724, 617, 1936, 2399, 4692, 848, 18

#### Checking summaries

In [27]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 51 µs, sys: 0 ns, total: 51 µs
Wall time: 53.6 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.23,0.42,0.29,0.06,999,0
1,-2.0,-1.95,test,0.27,0.43,0.25,0.05,-1,0
2,-2.0,-1.90,test,0.25,0.43,0.28,0.04,-1,0
3,-2.0,-1.85,test,0.30,0.40,0.27,0.03,-1,0
4,-2.0,-1.80,test,0.28,0.38,0.29,0.05,-1,0
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.39,0.21,0.06,0.34,-1,-1
6557,2.0,1.85,test,0.39,0.16,0.08,0.37,-1,-1
6558,2.0,1.90,test,0.31,0.22,0.09,0.38,-1,2


In [28]:
ham_summary.describe()

,t1,t2,-1,0,1,2,phase,pred_phase
count,6561.000000,6.561000e+03,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000,6561.000000
mean,0.000000,-4.331917e-18,0.196016,0.311168,0.308589,0.184228,28.045877,0.477061
std,1.169134,1.169134e+00,0.363242,0.433383,0.436459,0.358993,163.557011,1.020730
min,-2.000000,-2.000000e+00,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,-1.000000,-1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000e+00,0.000000,0.010000,0.010000,0.000000,1.000000,0.000000
75%,1.000000,1.000000e+00,0.130000,0.970000,0.980000,0.060000,1.000000,1.000000
max,2.000000,2.000000e+00,1.000000,1.000000,1.000000,1.000000,999.000000,2.000000


In [29]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  656100


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,-1,test
2,0,999,-1,test
3,0,999,-1,test
4,0,999,2,test
...,...,...,...,...
656095,6560,999,-1,test
656096,6560,999,-1,test
656097,6560,999,-1,test
656098,6560,999,1,test


#### Checking accuracies

In [30]:
simulation.accuracy

{'eigenvector_train': 0.999757334299167,
 'eigenvector_val': 0.9667028985507247,
 'eigenvector_test': 0.6505704307334109,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9981884057971014,
 'hamiltonian_test': 0.8766006984866124}

#### Checking data stored in  memory

In [31]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [32]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [33]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}

In [34]:
#simulation.fourier_matrix

In [35]:
#simulation.fourier_matrix.shape

In [36]:
#simulation.fourier_matrix[:,0]

In [37]:
#simulation.fourier_matrix[:,1]

In [38]:
#np.exp(-1j*2*np.pi*3/100)

In [39]:
#for i in range(1,50):
#    print("\n")
#    print(i)
#    print(np.sum((np.real(simulation.fourier_matrix[:,i]) - np.real(simulation.fourier_matrix[:,-i]))**2))
#    print(np.sum((np.imag(simulation.fourier_matrix[:,i])+ np.imag(simulation.fourier_matrix[:,-i]))**2))

In [40]:
#print(simulation.fourier_matrix[:,0])

In [41]:
#print(simulation.fourier_matrix[:,50])